## Regression Analysis of Post-Mortem Interval and Gene Expression with Model Stability Comparison

This Python script investigates how the expression levels of a key set of genes change with post-mortem interval (PMI). We model this relationship by comparing linear regression and kernel regression approaches, and we evaluate the stability of the models using stratified 5-fold cross-validation.

The script begins by loading the necessary data. Then, for each gene, we fit both linear regression and kernel regression models to analyze the relationship between gene expression and PMI. To assess the reliability of these models, we employ 5-fold cross-validation, ensuring model performance is evaluated on different subsets of the data.

Finally, we analyze the stability of the linear regression model's parameters (slope and intercept) across the different cross-validation folds by calculating the coefficient of variation and sign consistency. By comparing the results of linear regression and kernel regression, we aim to better understand the nature of the relationship between gene expression and PMI and to identify genes with stable and reliable correlations, which could be valuable for biomarker research.

In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

class LinearRegressionModel:
    def __init__(self):
        self.model = LinearRegression()

    def fit(self, X, y):
        X = np.asarray(X).reshape(-1, 1)
        y = np.asarray(y).ravel()
        self.model.fit(X, y)
        return self

    def predict(self, X):
        X = np.asarray(X).reshape(-1, 1)
        return self.model.predict(X)

    def get_params(self):
        # Return slope and intercept
        slope = float(self.model.coef_[0])
        intercept = float(self.model.intercept_)
        return slope, intercept

def compute_stability(df_cv):
    mean = df_cv.mean(axis=1)
    std  = df_cv.std(axis=1)
    cv   = std / mean.abs()
    sign_consistency = df_cv.apply(
        lambda row: np.mean(np.sign(row) == np.sign(row.mean())),
        axis=1
    )
    return pd.DataFrame({'cv': cv, 'sign_consistency': sign_consistency})

def main():
    # 1. Load data
    expr = pd.read_csv("pseudobulk_data.csv").rename(columns={"Unnamed: 0": "donor_id"})
    meta = pd.read_csv("meta_extracted.csv").rename(columns={"Donor ID": "donor_id"})
    sig  = pd.read_csv("significant_genes.csv")["Gene"].tolist()  # 73 genes

    # 2. Merge data and subset to the significant genes
    df = expr.merge(meta[['donor_id','PMI']], on='donor_id', how='inner')
    genes = [g for g in sig if g in df.columns]
    X = df['PMI'].values.reshape(-1,1)
    Y = df[genes].values  # shape: (n_samples, 73)

    # 3. Fit simple linear regression per gene (using the full dataset)
    slopes, intercepts = [], []
    for i in tqdm(range(Y.shape[1]), desc="Fitting linear regression"):
        model = LinearRegressionModel().fit(X, Y[:, i])
        m, b = model.get_params()
        slopes.append(m)
        intercepts.append(b)
    params_df = pd.DataFrame({'Gene': genes, 'Slope': slopes, 'Intercept': intercepts})
    params_df.to_csv("linear_params_73genes.csv", index=False)

    # 4. Stratified 5-fold CV on PMI bins
    bins = pd.qcut(X.flatten(), q=5, labels=False)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    folds = [f"Fold{i+1}" for i in range(5)]

    df_cv_slope = pd.DataFrame(index=genes, columns=folds, dtype=float)
    df_cv_int   = pd.DataFrame(index=genes, columns=folds, dtype=float)
    df_cv_mse   = pd.DataFrame(index=genes, columns=folds, dtype=float)

    for gi, gene in enumerate(genes):
        y = Y[:, gi].reshape(-1,1)
        for fold, (train_idx, test_idx) in enumerate(skf.split(X, bins)):
            Xtr, Xte = X[train_idx], X[test_idx]
            ytr, yte = y[train_idx], y[test_idx]
            model = LinearRegressionModel().fit(Xtr, ytr)
            m, b = model.get_params()
            y_pred = model.predict(Xte)
            mse = np.mean((yte.flatten() - y_pred)**2)
            df_cv_slope.at[gene, folds[fold]] = m
            df_cv_int.at[gene,   folds[fold]] = b
            df_cv_mse.at[gene,   folds[fold]] = mse

    df_cv_slope.to_csv("cv_slope_73genes.csv")
    df_cv_int.to_csv("cv_intercept_73genes.csv")
    df_cv_mse.to_csv("cv_mse_73genes.csv")

    # 5. Compute stability and filter stable genes
    stab_slope = compute_stability(df_cv_slope)
    stab_int   = compute_stability(df_cv_int)

    stable_slope = stab_slope[(stab_slope.cv < 0.4) & (stab_slope.sign_consistency > 0.9)]
    stable_int   = stab_int[(stab_int.cv < 0.4)   & (stab_int.sign_consistency > 0.9)]
    stable_both  = sorted(set(stable_slope.index) & set(stable_int.index))

    total = len(genes)
    print(f"Total genes: {total}")
    print(f"Stable Slopes:    {len(stable_slope)} ({len(stable_slope)/total:.2%})")
    print(f"Stable Intercepts:{len(stable_int)} ({len(stable_int)/total:.2%})")
    print(f"Stable both:      {len(stable_both)} ({len(stable_both)/total:.2%})")

    # 6. Save lists of stable genes
    stable_slope.index.to_series().to_frame("Gene")\
        .to_csv("stable_slope_73.csv", index=False)
    stable_int.index.to_series().to_frame("Gene")\
        .to_csv("stable_intercept_73.csv", index=False)
    pd.Series(stable_both, name="Gene").to_frame()\
        .to_csv("stable_both_73.csv", index=False)

if __name__ == "__main__":
    main()

Fitting linear regression: 100%|██████████| 73/73 [00:00<00:00, 1708.56it/s]


Total genes: 73
Stable Slopes:    73 (100.00%)
Stable Intercepts:73 (100.00%)
Stable both:      73 (100.00%)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from tqdm import tqdm

class KernelRegressionModel:
    def __init__(self, kernel='rbf', alpha=1.0, gamma=None):
        self.kernel = kernel
        self.alpha = alpha
        self.gamma = gamma
        self.model = KernelRidge(kernel=kernel, alpha=alpha, gamma=gamma)

    def fit(self, X, y):
        X = np.asarray(X).reshape(-1,1)
        y = np.asarray(y).ravel()
        if self.gamma is None:
            param_grid = {'alpha': [0.01,0.1,1,10], 'gamma': [0.001,0.01,0.1,1.0]}
            gs = GridSearchCV(KernelRidge(kernel=self.kernel),
                              param_grid, cv=min(5,len(X)),
                              scoring='neg_mean_squared_error')
            try:
                gs.fit(X, y)
                self.model = gs.best_estimator_
                self.alpha, self.gamma = gs.best_params_['alpha'], gs.best_params_['gamma']
            except:
                self.model = KernelRidge(kernel=self.kernel,
                                         alpha=self.alpha, gamma=self.gamma)
                self.model.fit(X, y)
        else:
            self.model.fit(X, y)
        return self

    def predict(self, X):
        X = np.asarray(X).reshape(-1,1)
        return self.model.predict(X)

    def get_params(self):
        return self.alpha, self.gamma


# === stability metrics ===
def compute_stability(df_cv):
    mean = df_cv.mean(axis=1)
    std  = df_cv.std(axis=1)
    cv   = std / mean.abs()
    sign_consistency = df_cv.apply(
        lambda row: np.mean(np.sign(row) == np.sign(row.mean())),
        axis=1
    )
    return pd.DataFrame({'cv': cv, 'sign_consistency': sign_consistency})


# === main pipeline ===
def main():
    # 1. Load data
    expr = pd.read_csv("pseudobulk_data.csv").rename(columns={"Unnamed: 0":"donor_id"})
    meta = pd.read_csv("meta_extracted.csv").rename(columns={"Donor ID":"donor_id"})
    sig  = pd.read_csv("significant_genes.csv")["Gene"].tolist()  # 73 genes

    # 2. Merge and subset to 73 sig genes
    df = expr.merge(meta[['donor_id','PMI']], on='donor_id', how='inner')
    genes = [g for g in sig if g in df.columns]
    X = df['PMI'].values.reshape(-1,1)
    Y = df[genes].values  # shape=(n_samples,73)

    # 3. Kernel regression per gene
    alphas, gammas = [], []
    for i in tqdm(range(Y.shape[1]), desc="Fitting kernel regression"):
        model = KernelRegressionModel()
        model.fit(X, Y[:,i])
        a,g = model.get_params()
        alphas.append(a); gammas.append(g)
    params_df = pd.DataFrame({'Gene':genes,'Alpha':alphas,'Gamma':gammas})
    params_df.to_csv("kernel_params_73genes.csv", index=False)

    # 4. Stratified 5-fold CV on PMI bins
    bins = pd.qcut(X.flatten(), q=5, labels=False)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    df_cv_a = pd.DataFrame(index=genes, columns=[f"Fold{i+1}" for i in range(5)], dtype=float)
    df_cv_g = pd.DataFrame(index=genes, columns=[f"Fold{i+1}" for i in range(5)], dtype=float)
    df_cv_m = pd.DataFrame(index=genes, columns=[f"Fold{i+1}" for i in range(5)], dtype=float)

    for gi, gene in enumerate(genes):
        gene_vals = Y[:,gi].reshape(-1,1)
        for fold,(train_idx,test_idx) in enumerate(skf.split(X, bins)):
            Xtr, Xte = X[train_idx], X[test_idx]
            ytr, yte = gene_vals[train_idx], gene_vals[test_idx]
            model = KernelRegressionModel().fit(Xtr, ytr)
            a,g = model.get_params()
            pred = model.predict(Xte)
            mse = np.mean((yte.flatten()-pred)**2)
            df_cv_a.at[gene, f"Fold{fold+1}"] = a
            df_cv_g.at[gene, f"Fold{fold+1}"] = g
            df_cv_m.at[gene, f"Fold{fold+1}"] = mse

    df_cv_a.to_csv("cv_alphas_73genes.csv")
    df_cv_g.to_csv("cv_gammas_73genes.csv")
    df_cv_m.to_csv("cv_mse_73genes.csv")

    # 5. Compute stability & filter stable genes
    stab_a = compute_stability(df_cv_a)
    stab_g = compute_stability(df_cv_g)

    stable_a = stab_a[(stab_a.cv < 0.5) & (stab_a.sign_consistency > 0.9)]
    stable_g = stab_g[(stab_g.cv < 0.5) & (stab_g.sign_consistency > 0.9)]
    stable_both = sorted(set(stable_a.index)&set(stable_g.index))

    total = len(genes)
    print(f"Total genes: {total}")
    print(f"Stable Alpha genes: {len(stable_a)} ({len(stable_a)/total:.2%})")
    print(f"Stable Gamma genes: {len(stable_g)} ({len(stable_g)/total:.2%})")
    print(f"Stable both:       {len(stable_both)} ({len(stable_both)/total:.2%})")

    # 6. Save stable lists
    stable_a.index.to_series().to_frame("Gene").to_csv("stable_alpha_73.csv", index=False)
    stable_g.index.to_series().to_frame("Gene").to_csv("stable_gamma_73.csv", index=False)
    pd.Series(stable_both, name="Gene").to_frame().to_csv("stable_both_73.csv", index=False)


if __name__ == "__main__":
    main()

Fitting kernel regression: 100%|██████████| 73/73 [00:06<00:00, 11.27it/s]


Total genes: 73
Stable Alpha genes: 35 (47.95%)
Stable Gamma genes: 14 (19.18%)
Stable both:       11 (15.07%)
